<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Static-Schedule" data-toc-modified-id="Static-Schedule-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Static Schedule</a></span><ul class="toc-item"><li><span><a href="#Homogeneous-Exponential-Case" data-toc-modified-id="Homogeneous-Exponential-Case-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Homogeneous Exponential Case</a></span></li><li><span><a href="#Heterogeneous-Exponential-Case" data-toc-modified-id="Heterogeneous-Exponential-Case-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Heterogeneous Exponential Case</a></span></li><li><span><a href="#Phase-Type-Case" data-toc-modified-id="Phase-Type-Case-1.3"><span class="toc-item-num">1.3&nbsp;&nbsp;</span>Phase-Type Case</a></span><ul class="toc-item"><li><span><a href="#Phase-Type-Fit" data-toc-modified-id="Phase-Type-Fit-1.3.1"><span class="toc-item-num">1.3.1&nbsp;&nbsp;</span>Phase-Type Fit</a></span></li><li><span><a href="#Weighted-Erlang-Distribution" data-toc-modified-id="Weighted-Erlang-Distribution-1.3.2"><span class="toc-item-num">1.3.2&nbsp;&nbsp;</span>Weighted Erlang Distribution</a></span></li><li><span><a href="#Hyperexponential-Distribution" data-toc-modified-id="Hyperexponential-Distribution-1.3.3"><span class="toc-item-num">1.3.3&nbsp;&nbsp;</span>Hyperexponential Distribution</a></span></li></ul></li></ul></li></ul></div>

# Static Schedule
_Roshan Mahes, Michel Mandjes, Marko Boon_

In this notebook we determine static schedules $t_1,\dots,t_n$ that minimize the following cost function:
\begin{align*}
\omega \sum_{i=1}^{n}\mathbb{E}I_i + (1 - \omega)\sum_{i=1}^{n}\mathbb{E}W_i,\quad \omega\in(0,1),
\end{align*}
where $I_i$ and $W_i$ are the expected idle and waiting time associated to client $i$, respectively. We assume that the service tasks $B_1,\dots,B_n$ are independent and solve the problem assuming different types of distributions.

The following packages are required:

In [83]:
# math
import numpy as np
import scipy
import math
from scipy.stats import binom, erlang, poisson
from scipy.optimize import minimize

# web scraping
from urllib.request import urlopen
from bs4 import BeautifulSoup as soup
import pandas as pd

# plotting
import plotly.graph_objects as go
import plotly.express as px
from itertools import cycle

## Homogeneous Exponential Case

In the first case, we assume $B_1,\dots,B_n \stackrel{i.i.d.}{\sim} B \stackrel{d}{=} \text{Exp}(\mu)$ for some $\mu > 0$. In our thesis, we have determined a recursive procedure. We state the result.

<div class="alert alert-warning">
<b>Corollary 2.5.</b>
For arrival time $t$ we have, with $X_t \sim \text{Pois}(\mu t)$ and $\ell = 2,\dots,k+1$,
\begin{align*}
p_{k1}(t) = \mathbb{P}(X_t\geq k),\quad 
p_{k\ell}(t) = \mathbb{P}(X_t = k-\ell+1).
\end{align*}
</div>

<div class="alert alert-warning">
<b>Proposition 2.7.</b>
Let $X_t \sim \text{Pois}(\mu t)$. Then
\begin{align*}
f_k(t) &= t\mathbb{P}(X_t\geq k) - \frac{k}{\mu}\mathbb{P}(X_t\geq k+1), \\
g_k(t) &= \frac{k(k-1)}{2\mu}\mathbb{P}(X_t\geq k+1) + (k-1)t\mathbb{P}(X_t\leq k-1) - \frac{\mu t^2}{2}\mathbb{P}(X_t\leq k-2).
\end{align*}
</div>

<div class="alert alert-warning">
<b>Theorem 2.8.</b>
Let $p_{k\ell}(t)$ and $f_k(t)$ and $g_{k}(t)$ be given by Corollary 2.5 and Proposition 2.7, and let $N_s$ denote the number of clients at time $s\geq 0$. Suppose that the clients are scheduled at (strictly increasing) times $t_1 = 0, t_2,\dots,t_n$. Then, for $i=1,\dots,n$,
\begin{align*}
\mathbb{E}I_i = \sum_{k=1}^{i-1}f_k(x_{i})\mathbb{P}(N_{t_{i-1}} = k),\quad
\mathbb{E}W_i = \sum_{k=1}^{i-1}g_k(x_{i})\mathbb{P}(N_{t_{i-1}} = k),
\end{align*}
where the following recursion holds: $\mathbb{P}(N_{t_1} = 1) = 1$ and for $i=2,\dots,n$ and $j = 1,\dots,i$,
\begin{align*}
\mathbb{P}(N_{t_i} = j) = \sum_{k=\max\{1,j-1\}}^{i-1}p_{kj}(x_{i})\mathbb{P}(N_{t_{i-1}} = k).
\end{align*}
</div>

We have implemented the formulas as follows.

In [84]:
def compute_probs_N_hom(t,mu=1):
    """
    Computes P(N_ti = j) for i=1,...,n and j=1,...,i.
    """
    
    n = len(t)
    probs = [[None for i in range(n)] for j in range(n)]
    probs[0][0] = 1
    
    for i in range(2,n+1):
        
        x = t[i-1] - t[i-2]
        
        # j = 1
        probs[i-1][0] = 0
        for l in range(1,i):
            probs[i-1][0] += (1 - poisson.cdf(l-1,mu*x)) * probs[i-2][l-1]

        # j = 2,...,i
        for j in range(2,i+1):
            probs[i-1][j-1] = 0
            for l in range(j-1,i):
                probs[i-1][j-1] += poisson.pmf(l-j+1,mu*x) * probs[i-2][l-1]
            
    return probs
    

def static_cost_hom(t,mu=1,omega=0.5):
    """
    Computes the costs given a schedule t=t_1,...,t_n.
    """
    
    n = len(t)
    
    # total expected waiting/idle time
    sum_EW, sum_EI = 0, 0
    probs = compute_probs_N_hom(t,mu)
    
    for i in range(2,n+1):
        for j in range(2,i+1):
            sum_EW += probs[i-1][j-1] * (j - 1) / mu
            
        for j in range(1,i):
            
            x = t[i-1] - t[i-2]
            f = (1 - poisson.cdf(j-1,mu*x)) * x - (1 - poisson.cdf(j,mu*x)) * j / mu
            sum_EI += f * probs[i-2][j-1]
            
    return omega * sum_EI + (1 - omega) * sum_EW

The optimal schedule can be found by minimizing the costs using the `scipy.minimize` function.

In [85]:
mu = 1
omega = 0.5
n = 10

optimization = minimize(static_cost_hom, range(n), args=(mu, omega),  bounds=(((0,0),) + (((0,None)),) * (n-1)))

print(f'Optimal Schedule: {optimization.x}')
print(f'Costs: {optimization.fun}')

Optimal Schedule: [ 0.          1.00600514  2.51512383  4.10465841  5.71585376  7.32585456
  8.91667095 10.46216439 11.90388492 13.02918991]
Costs: 4.69354313978878


Now we make a plot in which our schedules are obtained by web scraping the schedules from www.appointmentscheduling.info, a webapp created by Ruben Brokkelkamp.

In [86]:
mu = 1
n = 15

optimizations = []
costs = []

for omega in np.arange(0.1,1,0.1):
    
    url = f'http://www.appointmentscheduling.info/index.php?SCV=1&N={n}&omega={omega}&objFun=1'

    # opening up connection, grabbing the page
    uClient = urlopen(url)
    page_html = uClient.read()
    uClient.close()

    # html parsing
    page_soup = soup(page_html, "html.parser")
    table = page_soup.findAll("table", {"class": "bordered"})[1]

    # get appointment schedule
    df = pd.read_html(str(table))[0]
    schedule = df[df.columns[2]].values[:-2]
    
    optimizations += [schedule]
    costs += [static_cost_hom(schedule,mu,omega)]

In [88]:
palette = cycle(px.colors.sequential.algae)
inter_times = np.diff(optimizations)

fig = go.Figure()

for i in range(9):
    
    omega = 0.1 * (i + 1)
    fig.add_trace(go.Scatter(x=np.arange(2,n+1), y=inter_times[i],
                             name=f'${omega:.1f}$', marker_color=next(palette)))

fig.update_layout(
    template='plotly_white',
    title='$\\text{Static Schedule}\ (n=15)$',
    legend_title='$\\text{Omega}\ (\omega)$', 
    xaxis = {'title': '$\\text{Client Position}\ (i)$', 'range': [1.7, n + 0.3], 'dtick': 1},
    yaxis = {'title': '$\\text{Interarrival Time}\ (x_i)$', 'range': [0,3]}
)

fig.show()

print(f"costs: {[f'{costs[i]:.2f}' for i in range(len(costs))]}")

costs: ['3.51', '5.33', '6.51', '7.23', '7.55', '7.47', '6.94', '5.85', '3.92']


## Heterogeneous Exponential Case

Now we consider the case that the service tasks $B_i$ are independent and _heterogeneous exponentially_ distributed, i.e. $B_i \sim \text{Exp}(\mu_i)$, $i=1,\dots,n$. For ease we assume that all $\mu_i$ are distinct, i.e., $\mu_i \neq \mu_j$ for $i,j = 1,\dots,n$, $i\neq j$, but the case that some of the $\mu_i$ coincide can be considered analogously. We obtain the following result.

<div class="alert alert-warning">
<b>Lemma 2.12.</b>
For $k=1,\dots,n$ and $\ell=0,\dots,n-k$, we can write the density $\varphi_{k\ell}$ as
\[
\varphi_{k\ell}(s) := \mathbb{P}\left(\sum_{j=k}^{k+\ell}B_j \in\mathrm{d}s\right)
= \sum_{j=k}^{k+\ell}c_{k\ell j}e^{-\mu_j s},\quad s \geq 0.
\]
The coefficients $c_{k\ell j}$ are given recursively through $c_{k0k} = \mu_k$ and
\[
c_{k,\ell+1,j} = c_{k\ell j}\frac{\mu_{k+\ell+1}}{\mu_{k+\ell+1} - \mu_j}\quad \text{for}\ j = k,\dots,k+\ell,\quad c_{k,\ell+1,k+\ell+1} = \sum_{j=k}^{k+\ell}c_{k\ell j}\frac{\mu_{k+\ell+1}}{\mu_j - \mu_{k+\ell+1}}.
\]
</div>

<div class="alert alert-warning">
<b>Proposition 2.16.</b>
For $i=1,\dots,n-1$, $k=1,\dots,i$, $\ell = 2,\dots,k+1$ and $t\geq 0$,
\[
p_{k1,i}(t) = 1 - \sum_{\ell=2}^{k+1}p_{k\ell,i}(t),\quad
p_{k\ell,i}(t) = \frac{\varphi_{i-k+1,k-\ell+1}(t)}{\mu_{i-\ell+2}}.
\]
</div>

<div class="alert alert-warning">
<b>Proposition 2.17.</b>
For $i=1,\dots,n-1$ and $k=1,\dots,i$,
\begin{align*}
f_{k,i}(t) = t - \sum_{j=i-k+1}^{i}\frac{c_{i-k+1,k-1,j}}{\mu_j}\psi_{j}(t),
\quad
g_{k,i}(t) = \sum_{\ell=0}^{k-1}(k-\ell-1)\sum_{j=i-k+1}^{i-k+\ell+1}\frac{c_{i-k+1,\ell,j}}{\mu_{i-k+\ell+1}}\psi_{j}(t),
\end{align*}
with $\psi_{j}(t) = (1 - e^{-\mu_j t})/\mu_j$.
</div>

<div class="alert alert-warning">
<b>Theorem 2.18.</b>
Let $N_s$ denote the number of clients at time $s\geq 0$. Suppose that the clients are scheduled at (strictly increasing) times $t_1 = 0, t_2,\dots,t_n$. Then, for $i=1,\dots,n$,
\[
\mathbb{E}I_i = \sum_{k=1}^{i-1}f_{k,i-1}(x_{i})\mathbb{P}(N_{t_{i-1}} = k),\quad
\mathbb{E}W_i = \sum_{k=1}^{i-1}g_{k,i-1}(x_{i})\mathbb{P}(N_{t_{i-1}} = k),
\]
where the following recursion holds: $\mathbb{P}(N_{t_1} = 1) = 1$ and for $i=2,\dots,n$ and $j = 1,\dots,i$,
\begin{align*}
\mathbb{P}(N_{t_i} = j) = \sum_{k=\max\{1,j-1\}}^{i-1}p_{kj,i-1}(x_{i})\mathbb{P}(N_{t_{i-1}} = k).
\end{align*}
</div>

These formulas lead to the following implementation.

In [89]:
# helper functions
def c(k,l,j,mu):
    """
    Computes the weights c of phi recursively (Lemma 2.23).
    """

    # storage indices
    k_, l_, j_ = k - 1, l, j - 1
    
    if c_stored[k_][l_][j_] != None:
        pass
    elif k == j and not l:
        c_stored[k_][l_][j_] = mu[k_]
    elif l:
        if j >= k and j < k + l:
            c_stored[k_][l_][j_] = c(k,l-1,j,mu) * mu[k_+l_] / (mu[k_+l_] - mu[j-1])
        elif k + l == j:
            c_stored[k_][l_][j_] = np.sum([c(k,l-1,m,mu) * mu[j-1] / (mu[m-1] - mu[j-1]) for m in range(k,k+l)])
    
    return c_stored[k_][l_][j_]

def phi(k,l,s,mu):
    return np.sum([c(k,l,j,mu) * np.exp(-mu[j-1] * s) for j in range(k,k+l+1)])

def psi(j,t,mu):
    return (1 - np.exp(-mu[j-1] * t)) / mu[j-1]

# transition probabilities & cost function
def compute_probN_het(t,mu):
    """Computes P(N_ti = j) for i=1,...,n and j=1,...,i."""
    
    n = len(mu)
    p = np.zeros((n,n))
    p[0][0] = 1
    
    for i in range(2,n+1):
        
        x = t[i-1] - t[i-2]
        
        # j = 1
        for k in range(1,i):
            p[i-1][0] += np.sum([c(i-k,k-1,m,mu) * psi(m,x,mu) for m in range(i-k,i)]) * p[i-2][k-1]
        
        # j = 2,...,i
        for j in range(2,i+1):
            p[i-1][j-1] = np.sum([(phi(i-k,k-j+1,x,mu) / mu[i-j]) * p[i-2][k-1] for k in range(j-1,i)])
            
    return p

def static_cost_het(t,mu,omega):
    """Computes the cost of the optimal static schedule."""
    
    mu, n = np.array(mu), len(mu)
    EW, EI = np.zeros(n), np.zeros(n)    
    p = compute_probN_het(t,mu)
    
    for i in range(2,n+1):
        
        x = t[i-1] - t[i-2]
        EW[i-2] = np.sum([np.sum(1 / mu[i-j:i-1]) * p[i-1][j-1] for j in range(2,i+1)])
        
        for j in range(1,i):
            f = np.sum([c(i-j,j-1,m,mu) * (x - psi(m,x,mu)) / mu[m-1] for m in range(i-j,i)])
            EI[i-2] += f * p[i-2][j-1]
            
    return omega * np.sum(EI) + (1 - omega) * np.sum(EW)


Again we draw some plots.

In [52]:
omega = 0.5
n = 15

mus_incr = np.linspace(0.5,1.5,n)
mus_list = [mus_incr, np.random.permutation(mus_incr), np.random.permutation(mus_incr),
                np.random.permutation(mus_incr), mus_incr[::-1]]
mus_list_names = ['increasing', 'permutation 1', 'permutation 2', 'permutation 3', 'decreasing']

schedules = []
costs = []

for mus in mus_list:
    

    c_stored = [[[None for j in range(n)] for l in range(n)] for k in range(n)]
    optimization = minimize(static_cost_het, range(n), args=(mus,omega), bounds=(((0,0),) + (((0,None)),) * (n-1)))

    print(f'mus: {mus}')
    print(f'omega: {omega}')
    print(f'Optimal Schedule: {optimization.x}')
    print(f'Costs: {optimization.fun}\n')
    
    schedules += [optimization.x]
    costs += [optimization.fun]

palette = cycle(px.colors.sequential.algae)
fig = go.Figure()

for i in range(len(mus_list)):
    
    fig.add_trace(go.Scatter(x=np.arange(1,n+1), y=schedules[i], name=mus_list_names[i], marker_color=next(palette)))

fig.update_layout(
    template='plotly_white',
    title='$\\text{Static Schedule}\ (n=15)$',
    legend_title="$\mu\in [0.5,1.5]$", 
    xaxis={'title': '$\\text{Client Position}\ (i)$', 'range': [0.7, n + 0.3], 'dtick': 1},
    yaxis={'title': '$\\text{Arrival Time}\ (t_i)$'}
)

fig.show()

mus: [0.5        0.57142857 0.64285714 0.71428571 0.78571429 0.85714286
 0.92857143 1.         1.07142857 1.14285714 1.21428571 1.28571429
 1.35714286 1.42857143 1.5       ]
omega: 0.5
Optimal Schedule: [ 0.          2.15073911  5.04572009  7.77554192 10.27611596 12.55935893
 14.64883471 16.56731391 18.33412803 19.96437054 21.46851419 22.85069389
 24.10522414 25.20144663 26.00329857]
Costs: 9.23276627587529

mus: [1.         0.64285714 1.21428571 1.14285714 1.28571429 0.71428571
 1.5        0.57142857 0.85714286 1.35714286 0.5        0.92857143
 0.78571429 1.42857143 1.07142857]
omega: 0.5
Optimal Schedule: [ 0.          0.91384661  3.40706694  4.86389005  6.34902486  7.53509231
  9.84296812 10.86289228 13.6190404  15.70558936 16.813415   19.912121
 21.67747379 23.62481664 24.47703148]
Costs: 8.858774018911545

mus: [1.5        0.71428571 1.14285714 0.78571429 1.07142857 1.28571429
 1.         1.42857143 0.64285714 0.5        0.57142857 1.21428571
 0.85714286 1.35714286 0.92857143]
ome

In [90]:
omega = 0.5
n = 15

mus_Delta = lambda Delta: np.linspace(1 - Delta/2,1 + Delta/2,n)
Deltas = np.arange(0.6,1.6,0.1)
mus_list = list(map(mus_Delta, Deltas))

schedules = []
costs = []

for mus in mus_list:

    c_stored = [[[None for j in range(n)] for l in range(n)] for k in range(n)]
    optimization = minimize(static_cost_het, range(n), args=(mus,omega), bounds=(((0,0),) + (((0,None)),) * (n-1)))

    print(f'mus: {mus}')
    print(f'omega: {omega}')
    print(f'Optimal Schedule: {optimization.x}')
    print(f'Costs: {optimization.fun}\n')
    
    schedules += [optimization.x]
    costs += [optimization.fun]

# plot arrival times
palette = cycle(px.colors.sequential.algae)
fig = go.Figure()

for i in range(len(mus_list)):
    
    fig.add_trace(go.Scatter(x=np.arange(1,n+1), y=schedules[i], name=f'{Deltas[i]:.1f}', marker_color=next(palette)))

fig.update_layout(
    template='plotly_white',
    title='$\\text{Static Schedule}\ (n=15)$',
    legend_title="$\\text{Delta}\ (\Delta)$", 
    xaxis={'title': '$\\text{Client Position}\ (i)$', 'range': [0.7, n + 0.3], 'dtick': 1},
    yaxis={'title': '$\\text{Arrival Time}\ (t_i)$'}
)

fig.show()

mus: [0.7        0.74285714 0.78571429 0.82857143 0.87142857 0.91428571
 0.95714286 1.         1.04285714 1.08571429 1.12857143 1.17142857
 1.21428571 1.25714286 1.3       ]
omega: 0.5
Optimal Schedule: [ 0.          1.4852629   3.6127861   5.74549957  7.80822508  9.78457281
 11.67204487 13.47230442 15.18747784 16.8193406  18.36805583 19.82820048
 21.18445761 22.39533074 23.30144401]
Costs: 8.241305275018528

mus: [0.65 0.7  0.75 0.8  0.85 0.9  0.95 1.   1.05 1.1  1.15 1.2  1.25 1.3
 1.35]
omega: 0.5
Optimal Schedule: [ 0.          1.60813616  3.88575719  6.14138374  8.29805254 10.34241966
 12.27583845 14.10339657 15.83063359 17.46158681 18.99847283 20.43738678
 21.76650945 22.94682737 23.82473258]
Costs: 8.433176031963018

mus: [0.6        0.65714286 0.71428571 0.77142857 0.82857143 0.88571429
 0.94285714 1.         1.05714286 1.11428571 1.17142857 1.22857143
 1.28571429 1.34285714 1.4       ]
omega: 0.5
Optimal Schedule: [ 0.          1.75605905  4.20792848  6.60214009  8.86229958 10

## Phase-Type Case

Our most general case consists of service time distributions constructed by convolutions and mixtures of exponential distributions, the so-called _phase-type distributions_.

### Phase-Type Fit

There are two special cases of phase-type distributions that are of particular interest: the weighted Erlang distribution and the hyperexponential distribution. The idea is to fit the first two moments of the real service-time distribution. The former distribution can be used to approximate any non-negative distribution with coefficient of variation below 1, whereas the latter can be used if this coefficient of variation is larger than 1. The parameters of the weighted Erlang and hyperexponential distribution are obtained with the following function.

In [7]:
def SCV_to_params(SCV, mean=1):
    
    # weighted Erlang case
    if SCV <= 1:
        K = math.floor(1/SCV)
        p = ((K + 1) * SCV - math.sqrt((K + 1) * (1 - K * SCV))) / (SCV + 1)
        mu = K + (1 - p) / mean
    
        return K, p, mu
    
    # hyperexponential case
    else:
        p = 0.5 * (1 + np.sqrt((SCV - 1) / (SCV + 1)))
        mu = 1 / mean
        mu1 = 2 * p * mu
        mu2 = 2 * (1 - p) * mu
        
        return p, mu1, mu2

In the following figure, we have plotted the density of the fitted phase-type distribution given $\mathbb{E}B = 1$ and $\mathbb{S}(B)$. It is clear that the density becomes more concentrated to the mean for lower values for $\mathbb{S}(B)$.

In [142]:
def density_WE(x, K, p, mu):
    return p * erlang.pdf(x, K, scale=1/mu) + (1 - p) * erlang.pdf(x, K+1, scale=1/mu)

def density_HE(x, p, mu1, mu2):
    return p * mu1 * np.exp(-mu1 * x) + (1 - p) * mu2 * np.exp(-mu2 * x)

palette = cycle(px.colors.sequential.algae)
fig = go.Figure()

SCV_list = list(np.arange(0.2,1.1,0.1)) + [1.5, 2.0, 5.0]
x = np.linspace(0,4,1001)

for SCV in SCV_list:
    if SCV <= 1:
        K, p, mu = SCV_to_params(SCV)
        f_x = density_WE(x, K, p, mu)
    else:
        p, mu1, mu2 = SCV_to_params(SCV)
        f_x = density_HE(x, p, mu1, mu2)
    
    fig.add_trace(go.Scatter(x=x, y=f_x, name=f'${SCV:.1f}$', marker_color=next(palette)))

fig.update_layout(
    template='plotly_white',
    title='Phase-Type Fit',
    legend_title='$\\text{SCV}$', 
    xaxis={'title': 'x', 'range': [0, 4], 'dtick': 0.5},
    yaxis={'title': 'density'}
)

fig.show()

In the following subsections we develop procedures for finding the optimal static schedule in the weighted Erlang case and the hyperexponential case, respectively.

### Weighted Erlang Distribution

In this case the service times $B_i$ are independent and distributed as $B$, where $B$ follows an $\text{Erlang}(K,\mu)$-distribution with probability $p\in [0,1]$, and an $\text{Erlang}(K+1,\mu)$-distribution with probability $1-p$ for some $K \in \mathbb{N}$. The memoryless property doesn't hold for the weighted Erlang distribution unless $K=1$ and $p=1$; however, each phase is $\text{Exp}(\mu)$-distributed. Given that a client has been in phase $y=1,\dots,K+1$ for time $u$, the remaining time that the client stays in this phase is still $\text{Exp}(\mu)$ distributed (due to the memoryless property). Hence, at each time point the relevant information (i.e., the state) is the number of clients in the system, and the phase the client in service is in. We end up with the following theorem, where the expressions for the transition probabilities and the functions $\bar{f}$ and $\bar{h}$ can be found in the thesis.

<div class="alert alert-warning">
<b>Theorem 2.27.</b>
Let $N_s$ denote the number of clients at time $s\geq 0$. Suppose that the clients are scheduled at (strictly increasing) times $t_1=0,t_2,\dots,t_n$. Then, for $i = 1,\dots,n$, with $x_i := t_{i} - t_{i-1}$,
\[
\mathbb{E}I_i = \sum_{k=1}^{i-1}\sum_{y=1}^{K+1}\bar{f}_{ky}(x_i)\mathbb{P}(N_{t_{i-1}} = k, Z_{t_{i-1}} = y),
\quad
\mathbb{E}W_i = \sum_{k=1}^{i}\sum_{y=1}^{K+1}\bar{h}_{ky}\mathbb{P}(N_{t_i} = k, Z_{t_i} = y),
\]
where the following recursion holds: $\mathbb{P}(N_{t_1} = 1, Z_{t_1} = 1) = 1$ and for $i = 1,\dots,n$, $j = 1,\dots,i$ and $z = 1,\dots,K+1$,
\[
\mathbb{P}(N_{t_i} = j, Z_{t_i} = z)
= \sum_{k=\max\{1,j-1\}}^{i-1}\sum_{y=1}^{K+1}p_{kj,yz}(x_i)\mathbb{P}(N_{t_{i-1}} = k, Z_{t_{i-1}} = y).
\]
</div>

We get the following implementation.

In [92]:
def trans_p(k,l,y,z,t,K,p,mu):
    
    # 1. No client has been served before time t.
    if l == k + 1:
        if y <= K and z <= K:
            return poisson.pmf(z-y, mu*t)
        elif y <= K and z == K+1:
            return (1-p) * poisson.pmf(K+1-y, mu*t)
        elif y == K+1 and z == K+1:
            return np.exp(-mu * t)
    
    # 2. All clients have been served before time t.
    elif l == 1 and z == 1:
        if y <= K:
            return sum([binom.pmf(m, k, 1-p) * erlang.cdf(t, k*K-y+m+1, scale=1/mu) for m in range(k+1)])
        elif y == K+1:
            return sum([binom.pmf(m, k-1, 1-p) * erlang.cdf(t, (k-1)*K+m+1, scale=1/mu) for m in range(k)])
    
    # 3. Some (but not all) clients have been served before time t.
    elif 2 <= l <= k:
        if y <= K:  
            if z <= K:
                return sum([binom.pmf(m, k-l+1, 1-p) * poisson.pmf((k-l+1)*K+m+z-y, mu*t) for m in range(k-l+2)])
            elif z == K+1:
                return (1-p) * sum([binom.pmf(m, k-l+1, 1-p) * poisson.pmf((k-l+1)*K+m+z-y, mu*t) for m in range(k-l+2)])
        
        elif y == K+1:
            if z <= K:
                return sum([binom.pmf(m, k-l, 1-p) * poisson.pmf((k-l)*K+m+z, mu*t) for m in range(k-l+1)])
            elif z == K+1:
                return (1-p) * sum([binom.pmf(m, k-l, 1-p) * poisson.pmf((k-l)*K+m+z, mu*t) for m in range(k-l+1)])
    
    # any other case is invalid
    return 0
    

In [93]:
def f(k,t,mu):
    return poisson.sf(k-1, mu*t) * t - poisson.sf(k, mu*t) * k / mu

def f_bar(t,k,y,K,p,mu):
    if y <= K:
        return sum([binom.pmf(m, k, 1-p) * f(k*K-y+m+1, t, mu) for m in range(k+1)])
    elif y == K+1:
        return sum([binom.pmf(m, k, 1-p) * f((k-1)*K+m+1, t, mu) for m in range(k)])

def h_bar(k,y,K,p,mu):
    if k == 1:
        return 0
    else:
        if y <= K:
            return ((k-1)*(K+1-p) + 1 - y) / mu
        elif y == K+1:
            return ((k-2)*(K+1-p) + 1) / mu

def compute_probs_we(t,K,p,mu):
    """
    Computes P(N_ti = j, Z_ti = z) for i=1,...,n, j=1,...,i and z=1,...,K.
    """
    
    n = len(t)
    probs = [[[None for z in range(K+1)] for j in range(i+1)] for i in range(n)]
    
    probs[0][0][0] = 1
    for z in range(2,K+2):
        probs[0][0][z-1] = 0
    
    for i in range(2,n+1):
        
        x_i = t[i-1] - t[i-2]
        
        for j in range(1,i+1):
            for z in range(1,K+2):
                probs[i-1][j-1][z-1] = 0

                for k in range(max(1,j-1),i):
                    for y in range(1,K+2):
                        probs[i-1][j-1][z-1] += trans_p(k,j,y,z,x_i,K,p,mu) * probs[i-2][k-1][y-1]
    return probs

    
def static_cost_we(t,K,p,mu,omega):
    """
    Computes the cost of a static schedule in the weighted Erlang case.
    """
    
    n = len(t)
    
    # total expected waiting/idle time
    sum_EW, sum_EI = 0, 0
    probs = compute_probs_we(t,K,p,mu)
    
    for i in range(2,n+1):
        
        # waiting time
        for k in range(2,i+1):
            for y in range(1,K+2):
                sum_EW += h_bar(k,y,K,p,mu) * probs[i-1][k-1][y-1]
        
        # idle time
        for k in range(1,i):
            for y in range(1,K+2):
                
                x_i = t[i-1] - t[i-2]
                sum_EI += f_bar(x_i,k,y,K,p,mu) * probs[i-2][k-1][y-1]
            
    return omega * sum_EI + (1 - omega) * sum_EW

We proceed by analyzing the second case, i.e., the hyperexponential case.

### Hyperexponential Distribution

In this case the service times $B_i$ are independent and distributed as $B$, where $B$ equals with probability $p\in [0,1]$ an exponentially distributed random variable with mean $\mu_1^{-1}$, and with probability $1-p$ an exponentially distributed random variable with mean $\mu_{2}^{-1}$. The following theorem can be found in the thesis, where the expressions for the transition probabilities and the functions $\bar{f}$ and $\bar{h}$ can be found in the thesis.

<div class="alert alert-warning">
<b>Theorem 2.30.</b>
Let $N_s$ denote the number of clients at time $s\geq 0$. Suppose that the clients are scheduled at (strictly increasing) times $t_1=0,t_2,\dots,t_n$. Then, for $i = 1,\dots,n$, with $x_i := t_{i} - t_{i-1}$,
\[
\mathbb{E}I_i = \sum_{k=1}^{i-1}\sum_{y=1}^{2}\bar{f}_{ky}(x_i)\mathbb{P}(N_{t_{i-1}} = k, Z_{t_{i-1}} = y),
\quad
\mathbb{E}W_i = \sum_{k=1}^{i}\sum_{y=1}^{2}\bar{h}_{ky}\mathbb{P}(N_{t_i} = k, Z_{t_i} = y),
\]
where the following recursion holds:
\[
\mathbb{P}(N_{t_1} = 1, Z_{t_1} = 1) = p,
\quad
\mathbb{P}(N_{t_1} = 1, Z_{t_1} = 2) = 1-p,
\]
and, for $i = 1,\dots,n$, $j = 1,\dots,i$ and $z \in \{1,2\}$,
\[
\mathbb{P}(N_{t_i} = j, Z_{t_i} = z)
= \sum_{k=\max\{1,j-1\}}^{i-1}\sum_{y=1}^{2}p_{kj,yz}(x_i)\mathbb{P}(N_{t_{i-1}} = k, Z_{t_{i-1}} = y).
\]
</div>

Below is our implementation.

In [122]:
def trans_p(k,l,y,z,t,p,mu1,mu2):
    
    # 1. No client has been served before time t.
    if l == k+1 and z == y:
        if y == 1:
            return np.exp(-mu1 * t)
        elif y == 2:
            return np.exp(-mu2 * t)
    
    # 2. All clients have been served before time t.
    elif l == 1:
        if y == 1:
            prob = sum([binom.pmf(m, k-1, p) * psi(t, m+1, k-1-m, mu1, mu2) for m in range(k)])
            if z == 1:
                return p * prob
            elif z == 2:
                return (1-p) * prob
        elif y == 2:
            prob = sum([binom.pmf(m, k-1, p) * psi(t, m, k-m, mu1, mu2) for m in range(k)])
            if z == 1:
                return p * prob
            elif z == 2:
                return (1-p) * prob
    
    # 3. Some (but not all) clients have been served before time t.
    elif 2 <= l <= k:
        if y == 1:
            prob_diff = sum([binom.pmf(m, k-l, p) * psi(t, m+1, k-l-m, mu1, mu2) for m in range(k-l+1)]) \
                            - sum([binom.pmf(m, k-l+1, p) * psi(t, m+1, k-l+1-m, mu1, mu2) for m in range(k-l+2)])
            if z == 1:
                return p * prob_diff
            elif z == 2:
                return (1-p) * prob_diff
        elif y == 2:
            prob_diff = sum([binom.pmf(m, k-l, p) * psi(t, m, k-l+1-m, mu1, mu2) for m in range(k-l+1)]) \
                            - sum([binom.pmf(m, k-l+1, p) * psi(t, m, k-l+2-m, mu1, mu2) for m in range(k-l+2)])
            if z == 1:
                return p * prob_diff
            elif z == 2:
                return (1-p) * prob_diff
    
    # any other case is invalid
    return 0


In [123]:
def zeta(alpha, t, k):
    
    if not k:
        return (np.exp(alpha * t) - 1) / alpha
    else:
        return ((t ** k) * np.exp(alpha * t) - k * zeta(alpha, t, k-1)) / alpha

def rho(t,m,k,mu1,mu2):
    
    if not k:
        return np.exp(-mu2 * t) * (mu1 ** m) / ((mu1 - mu2) ** (m + 1)) * erlang.cdf(t, m+1, scale=1/(mu1 - mu2))
    elif not m:
        return np.exp(-mu1 * t) * (mu2 ** k) / math.factorial(k) * zeta(mu1-mu2, t, k)
    else:
        return (mu1 * rho(t, m-1, k, mu1, mu2) - mu2 * rho(t, m, k-1, mu1, mu2)) / (mu1 - mu2)

def psi(t,m,k,mu1,mu2):
    
    if not m:
        return erlang.cdf(t, k, scale=1/mu2)
    else:
        return erlang.cdf(t, m, scale=1/mu1) - mu1 * sum([rho(t, m-1, i, mu1, mu2) for i in range(k)])
    
def sigma(t,m,k,mu1,mu2):
    
    if not k:
        return t * erlang.cdf(t, m, scale=1/mu1) - (m / mu1) * erlang.cdf(t, m+1, scale=1/mu1)
    elif not m:
        return t * erlang.cdf(t, k, scale=1/mu2) - (k / mu2) * erlang.cdf(t, k+1, scale=1/mu2)
    else:
        return (t - k / mu2) * erlang.cdf(t, m, scale=1/mu1) - (m / mu1) * erlang.cdf(t, m+1, scale=1/mu1) \
                    + (mu1 / mu2) * sum([(k - i) * rho(t, m-1, i, mu1, mu2) for i in range(k)])


In [124]:
def f_bar(t,k,y,p,mu1,mu2):
    
    if y == 1:
        return sum([binom.pmf(m, k-1, p) * sigma(t, m+1, k-1-m, mu1, mu2) for m in range(k)])
    elif y == 2:
        return sum([binom.pmf(m, k-1, p) * sigma(t, m, k-m, mu1, mu2) for m in range(k)])

def h_bar(k,y,mu1,mu2):
    
    if k == 1:
        return 0
    else:
        if y == 1:
            return (k-2) + (1/mu1)
        elif y == 2:
            return (k-2) + (1/mu2)

def compute_probs_hyp(t,p,mu1,mu2):
    """
    Computes P(N_ti = j, Z_ti = z) for i=1,...,n, j=1,...,i and z=1,2.
    """
    
    n = len(t)
    probs = [[[None for z in range(2)] for j in range(i+1)] for i in range(n)]
    
    probs[0][0][0] = p
    probs[0][0][1] = 1 - p
    
    for i in range(2,n+1):
        
        x_i = t[i-1] - t[i-2]
        
        for j in range(1,i+1):
            for z in range(1,3):
                probs[i-1][j-1][z-1] = 0

                for k in range(max(1,j-1),i):
                    for y in range(1,3):
                        probs[i-1][j-1][z-1] += trans_p(k,j,y,z,x_i,p,mu1,mu2) * probs[i-2][k-1][y-1]
    return probs

def static_cost_hyp(t,p,mu1,mu2,omega):
    """
    Computes the cost of a static schedule in the weighted Erlang case.
    """
    
    n = len(t)
    
    # total expected waiting/idle time
    sum_EW, sum_EI = 0, 0
    probs = compute_probs_hyp(t, p, mu1, mu2)
    
    for i in range(2,n+1):
        
        # waiting time
        for k in range(2,i+1):
            sum_EW += h_bar(k, 1, mu1, mu2) * probs[i-1][k-1][0] + h_bar(k, 2, mu1, mu2) * probs[i-1][k-1][1]
        
        # idle time
        for k in range(1,i):                
            x_i = t[i-1] - t[i-2]
            sum_EI += f_bar(x_i, k, 1, p, mu1, mu2) * probs[i-2][k-1][0] \
                    + f_bar(x_i, k, 2, p, mu1, mu2) * probs[i-2][k-1][1]
    
    return omega * sum_EI + (1 - omega) * sum_EW


We draw the following plot.

In [149]:
omega = 0.5
n = 15

schedules = []
costs = []

SCV_list = np.arange(0.5,1.51,0.1)

for SCV in SCV_list:
    
    url = f'http://www.appointmentscheduling.info/index.php?SCV={SCV:.1f}&N={n}&omega={omega}&objFun=1'

    # opening up connection, grabbing the page
    uClient = urlopen(url)
    page_html = uClient.read()
    uClient.close()

    # html parsing
    page_soup = soup(page_html, "html.parser")
    table = page_soup.findAll("table", {"class": "bordered"})[1]

    # get appointment schedule
    df = pd.read_html(str(table))[0]
    schedule = df[df.columns[2]].values[:-2]
    
    schedules += [schedule]
    # costs += [static_cost_hyp(schedule, K, p, mu, omega)]

# plot arrival times
palette = cycle(px.colors.sequential.algae)
fig = go.Figure()

for i in range(len(SCV_list)):
    
    fig.add_trace(go.Scatter(x=np.arange(1,n+1), y=schedules[i], name=f'{SCV_list[i]:.1f}', marker_color=next(palette)))

fig.update_layout(
    template='plotly_white',
    title='$\\text{Static Schedule}\ (n=15)$',
    legend_title="$\\text{SCV}\ (\mathbb{S}(B))$", 
    xaxis={'title': '$\\text{Client Position}\ (i)$', 'range': [0.7, n + 0.3], 'dtick': 1},
    yaxis={'title': '$\\text{Arrival Time}\ (t_i)$'}
)

fig.show()

# plot interarrival times
palette = cycle(px.colors.sequential.algae)
fig = go.Figure()

inter_times = np.diff(schedules)

for i in range(len(SCV_list)):
    
    fig.add_trace(go.Scatter(x=np.arange(2,n+1), y=inter_times[i], name=f'{SCV_list[i]:.1f}', marker_color=next(palette)))

fig.update_layout(
    template='plotly_white',
    title='$\\text{Static Schedule}\ (n=15)$',
    legend_title="$\\text{SCV}\ (\mathbb{S}(B))$", 
    xaxis={'title': '$\\text{Client Position}\ (i)$', 'range': [1.7, n + 0.3], 'dtick': 1},
    yaxis={'title': '$\\text{Interarrival Time}\ (x_i)$'}
)

fig.show()